# SMPD3 Enh3 Activity after Sox9 MO
## Fluorescence intensity measurements

Analyze whole mount fluorescent intensity measurements. This script with import raw measurements from Fiji, then normalize results based on electroporation efficiency.

In [1]:
# Import data handling and analysis packages
import os
import glob
import pandas as pd
from scipy import stats


# Import plotting packages
import iqplot
import bokeh.io
from bokeh.models import Span, Title
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

## Import source data

In [2]:
## Navigate to CSV path
path = os.path.abspath('')+'/raw_source_data/'
full_df = pd.DataFrame()
list_ = []

for file_ in glob.glob(path + "/*.csv"):         # For loop to bring in files and concatenate them into a single dataframe
    df = pd.read_csv(file_)
    
    if '_stack.csv' in file_:
            file_ = str(file_.rsplit('_', 1))
    
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]                      # Determine Image name from file name
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'],                   # Split values in Image name column
         df['Somites'], df['Mag']) = zip(*df['Image'].map(lambda x: x.split('_')))
    df['EmbID'] = df['ExptDate'] + '_' + df['Embryo']
    df['Fluor'] = df['Label'].map(lambda x: x.split(':')[0])                       # Split values in ROI label
    df['ROI'] = df['Label'].map(lambda x: x.split(':')[1])                          # Split values in ROI label
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,Min,Max,IntDen,RawIntDen,Image,ExptDate,Treatment,Stains,Embryo,Somites,Mag,EmbID,Fluor,ROI
0,1,SOX10:Background,64.927,14.898,0,72,9.673020e+02,4693.0,20221115_Sox9MO05mM_BF;Sox10;H2BRFP;Enh3GFP;Pa...,20221115,Sox9MO05mM,BF;Sox10;H2BRFP;Enh3GFP;Pax7,emb4,7ss,"10x', 'stack",20221115_emb4,SOX10,Background
1,2,SOX10:CntlArea,37087.482,410.354,0,4552,1.521900e+07,73837072.0,20221115_Sox9MO05mM_BF;Sox10;H2BRFP;Enh3GFP;Pa...,20221115,Sox9MO05mM,BF;Sox10;H2BRFP;Enh3GFP;Pax7,emb4,7ss,"10x', 'stack",20221115_emb4,SOX10,CntlArea
2,3,SOX10:ExptArea,16830.196,146.812,0,1713,2.470873e+06,11987779.0,20221115_Sox9MO05mM_BF;Sox10;H2BRFP;Enh3GFP;Pa...,20221115,Sox9MO05mM,BF;Sox10;H2BRFP;Enh3GFP;Pax7,emb4,7ss,"10x', 'stack",20221115_emb4,SOX10,ExptArea
3,4,H2BRFP:Background,64.927,10.946,0,43,7.106880e+02,3448.0,20221115_Sox9MO05mM_BF;Sox10;H2BRFP;Enh3GFP;Pa...,20221115,Sox9MO05mM,BF;Sox10;H2BRFP;Enh3GFP;Pax7,emb4,7ss,"10x', 'stack",20221115_emb4,H2BRFP,Background
4,5,H2BRFP:CntlArea,37087.482,273.755,0,4042,1.015290e+07,49258191.0,20221115_Sox9MO05mM_BF;Sox10;H2BRFP;Enh3GFP;Pa...,20221115,Sox9MO05mM,BF;Sox10;H2BRFP;Enh3GFP;Pax7,emb4,7ss,"10x', 'stack",20221115_emb4,H2BRFP,CntlArea


## Extract, organize, and analyse results

- Pull out raw results and calculate Corrected Total Cellular Fluorescence (CTCF) -> output = no_norm_results
- Normalize results to electroporation efficiency by calculating RFP/Citrine signal -> output = elec_norm_results
- Generate table with ratios of Experiment/Control values -> output = ratio_results

In [3]:
# Define control and experimental constructs and copy out raw data to analyze
cntl_construct = 'Control MO'
expt_construct = 'Sox9 MO'
data_df = full_df.copy()
 
# Initialize for final dataframe collection
no_norm_results_list = []

# Loop through fluorescent target channels:
fluor_list = full_df.Fluor.unique().tolist()
for fluor in fluor_list:
    df_fluor = (data_df.loc[data_df['Fluor'] == fluor][['ExptDate','Fluor','EmbID',
                                                       'Treatment','Somites','ROI','IntDen','Mean','Area']])
    # Loop through embryos:
    embryo_results_list = []          
    embryo_list = df_fluor.EmbID.unique().tolist()
    for embryo in embryo_list:
        df_embryo = df_fluor.loc[df_fluor['EmbID'] == embryo]
        cntl_intden = float(df_embryo.loc[df_embryo['ROI'] == 'CntlArea']['IntDen'])
        expt_intden = float(df_embryo.loc[df_embryo['ROI'] == 'ExptArea']['IntDen'])

        # Assemble output df from specific values in each embryo dataset, and append to building list of embryo dfs
        data = {'Fluor': [fluor, fluor], 'EmbID': [embryo, embryo] 
                ,'Treatment': [df_embryo.tail(1)['Treatment'].values[0], df_embryo.tail(1)['Treatment'].values[0]]
                ,'Somites': [df_embryo.tail(1)['Somites'].values[0], df_embryo.tail(1)['Somites'].values[0]]
                ,'ROI': [cntl_construct, expt_construct]
                ,'IntDen': [cntl_intden, expt_intden]
               }
        embryo_results_list.append(pd.DataFrame(data))

    # Combine embryo 
    no_norm_results_list.append(pd.concat(embryo_results_list, sort=False).reset_index().drop('index', axis=1))

# Assemble the final unnormalized results
no_norm_results = pd.concat(no_norm_results_list,sort=False).reset_index().drop('index', axis=1)

# Pull out separate channel DFs for electroporation normalization "ElecNorm" (Enh3 GFP/H2B-RFP)
h2brfp_results = no_norm_results.loc[no_norm_results['Fluor'] == 'H2BRFP'].reset_index().drop(['index'], axis=1)
elec_norm_results = no_norm_results.loc[no_norm_results['Fluor'] == 'Enh3GFP'].reset_index().drop(['index'], axis=1)
elec_norm_results['ElecNormIntDen'] = elec_norm_results['IntDen']/h2brfp_results['IntDen']

# Pull out ROIs to produce ratio results (Experiment/Control values) after normalizing to the mean of the control dataset
cntl_side_results = elec_norm_results.loc[elec_norm_results['ROI'] == cntl_construct].reset_index().drop(['index'], axis=1)
cntl_side_results['DoubleNormIntDen'] = cntl_side_results['ElecNormIntDen'] / cntl_side_results['ElecNormIntDen'].mean()
ratio_results = elec_norm_results.loc[elec_norm_results['ROI'] == expt_construct].reset_index().drop(['index'], axis=1)
ratio_results['DoubleNormIntDen'] = ratio_results['ElecNormIntDen'] / cntl_side_results['ElecNormIntDen'].mean()
elec_norm_results = pd.concat([cntl_side_results,ratio_results]) # Generate new elec_norm_results 
ratio_results['RatioDoubleNormIntDen'] = ratio_results['DoubleNormIntDen']/cntl_side_results['DoubleNormIntDen']

ratio_results.head(10)

,Fluor,EmbID,Treatment,Somites,ROI,IntDen,ElecNormIntDen,DoubleNormIntDen,RatioDoubleNormIntDen
0,Enh3GFP,20221115_emb4,Sox9MO05mM,7ss,Sox9 MO,271844.743,0.110053,0.403438,0.258898
1,Enh3GFP,20221115_emb5,Sox9MO05mM,7ss,Sox9 MO,220762.397,0.078637,0.288272,0.424603
2,Enh3GFP,20221115_emb12,Sox9MO05mM,7ss,Sox9 MO,148451.133,0.043319,0.158799,0.204860
3,Enh3GFP,20221115_emb2,Sox9MO05mM,8ss,Sox9 MO,911750.828,0.135061,0.495113,0.536409
4,Enh3GFP,20221115_emb1,Sox9MO05mM,6ss,Sox9 MO,154761.992,0.112028,0.410677,0.385751


## Plot ratio values

- Plot measured vallues as parallel coordinate plot showing control vs experimental sides
- Plot stripbox plot with the corresponding ratio results
- Perform two-tailed paired t test to determine if we can reject the null hypothesis

In [4]:
################### Isolate data for analysis ###################
# Annotate data further to plot 
cntl_construct = 'Control MO'
expt_construct = 'Sox9 MO'
metric_parcoord = 'DoubleNormIntDen'
metric_ratio = 'RatioDoubleNormIntDen'
dataset_to_plot_parcoord = elec_norm_results 
dataset_to_plot_ratio = ratio_results

# Target to parse:
target = ['Enh3GFP']
stages = ['6ss','7ss','8ss', '9ss', '10ss']

# Pull out only cells and treaments of interest, and rename ROIs with the appropriate constructs
df_parcoord = dataset_to_plot_parcoord.loc[dataset_to_plot_parcoord['Fluor'].isin(target)].copy()
df_parcoord.replace(to_replace = {'Cntl': cntl_construct, 'Expt': expt_construct}, inplace=True)
df_parcoord = df_parcoord.loc[df_parcoord['Somites'].isin(stages)].copy()

df_ratio = dataset_to_plot_ratio.loc[dataset_to_plot_ratio['Fluor'].isin(target)].copy()
df_ratio = df_ratio.loc[df_ratio['Somites'].isin(stages)].copy()
df_ratio.replace(to_replace = {'Sox9MO05mM': expt_construct}, inplace=True)

################### Plot as parallel coordinate plot ###################
# Plot as strip plot
p1 = iqplot.strip(
                # Data to plot
                data=df_parcoord
                ,q=metric_parcoord, q_axis='y'
                ,cats='ROI', parcoord_column='EmbID'
                # Plot customizations
                ,marker_kwargs=dict(alpha=1, size=9, color='dimgray', line_color='white')
                ,parcoord_kwargs=dict(line_width=1,color='gray')
#                 ,show_legend=True   
#                 ,y_range=(-2,2)
                ,frame_height = 400, frame_width = 300
                ,x_axis_label='Treatment'
                ,tooltips=[("Embryo", "@EmbID"), ]
              )
# Final customizations
p1.axis.axis_label_text_font_size = '16px'
p1.axis.major_label_text_font_size = '16px'
p1.axis.axis_label_text_font_style = 'bold'
p1.xaxis.major_label_text_font_style = 'italic'
p1.legend.location = 'top_right'

################### Plot as stripbox plot ###################
# Build Stripbox plot
p2 = iqplot.stripbox(
                # Data to plot
                data=df_ratio
                ,q=metric_ratio, q_axis='y'
                ,cats='Treatment' 
                # Plot customizations
                ,spread='swarm' ,jitter_kwargs=dict(width=0.4)
                ,marker_kwargs=dict(alpha=1, size=10, color='dimgray', line_color='white')
                ,box_kwargs=dict(line_color='black', line_width=1.5,fill_color='white', fill_alpha=1)
                ,whisker_kwargs=dict(line_color='black', line_width=1.5)
                ,median_kwargs=dict(line_color='black', line_width=2)
                ,top_level='strip'
#                 ,show_legend=False
                ,y_range=(0,1.5)
                ,min_data=3
                ,frame_height = 400, frame_width = 150
                ,x_axis_label='Treatment'
                ,y_axis_label='GFP Expression Relative to Control'
                ,tooltips=[("Embryo", "@EmbID"), ]
)
# Final customizations
p2.axis.axis_label_text_font_size = '16px'
p2.axis.major_label_text_font_size = '16px'
p2.axis.axis_label_text_font_style = 'bold'
p2.xaxis.major_label_text_font_style = 'italic'
vline = Span(location=1,dimension='width', level='underlay',
             line_color='darkgray',line_width=2)
p2.add_layout(vline)

################### Display plots ###################
show(row(p1, p2))

################### Perform statistical analysis ###################
# Perform Paired t test 
cntl = df_parcoord.loc[df_parcoord['ROI'] == cntl_construct][metric_parcoord]
expt = df_parcoord.loc[df_parcoord['ROI'] == expt_construct][metric_parcoord]
ttest = stats.ttest_rel(cntl,expt)

# Display test results
print('Paired t-test results: \n\t\t statistic = ' + str(ttest[0]) + 
    '\n\t\t p-value = ' + str(ttest[1]))
print('n = ' + str(len(cntl)) + ' embryos')

/Users/michaelpiacentino/opt/anaconda3/lib/python3.7/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


Paired t-test results: 
		 statistic = 4.752941389035284
		 p-value = 0.008951515058724938
n = 5 embryos


In [5]:
expt_side = 'Sox9 MO'
cntl_side = 'Control MO'
output_file = 'Enh3_Sox9MO_Data'

# Prepare data for exporting to Prism
df_prism = df_parcoord.pivot(index=('Fluor','EmbID','Treatment','Somites'), 
                             columns='ROI', values='DoubleNormIntDen').reset_index()
df_prism['Norm '+ cntl_side] = df_prism[cntl_side]/df_prism[cntl_side].mean()
df_prism['Norm '+ expt_side] = df_prism[expt_side]/df_prism[cntl_side].mean()
df_prism['Norm Ratio'] = (df_prism[expt_side]/df_prism[cntl_side])

df_prism.to_csv(output_file+'.csv')